## Initial conditions/ setting matrices A, B, C


In [17]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags, csr_matrix, diags
from scipy.integrate import solve_ivp
from scipy.linalg import lu, solve, solve_triangular
from scipy.sparse.linalg import gmres, bicgstab, spsolve, spilu
from scipy.fftpack import fft2, ifft2
import imageio
from IPython.display import Image
import imageio.v2 as imageio 
from io import BytesIO
import os
import time
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from matplotlib.colors import LinearSegmentedColormap

### creating matrices

m = 128 # N value in x and y directions
n = m * m # total size of matrix
deltax = 20/m
deltaS = (20/m) **2

e0 = np.zeros((n, 1)) # vector of zeros
e1 = np.ones((n, 1)) # vector of ones

e2 = np.copy(e1) # copy the one vector
e4 = np.copy(e0) # copy the zero vector

for j in range(1, m+1):
    e2[m*j-1] = 0 # overwrite every m^th value with zero
    e4[m*j-1] = 1 # overwirte every m^th value with one
    
# Shift to correct positions
e3 = np.zeros_like(e2)
e3[1:n] = e2[0:n-1]
e3[0] = e2[n-1]

e5 = np.zeros_like(e4)
e5[1:n] = e4[0:n-1]
e5[0] = e4[n-1]

diagonals = [e1.flatten(), e1.flatten(), e5.flatten(),
e2.flatten(), -4 * e1.flatten(), e3.flatten(),
e4.flatten(), e1.flatten(), e1.flatten()]
offsets = [-(n-m), -m , -(m-1), -1, 0, 1, m - 1, m , (n-m)]
matA = spdiags(diagonals, offsets, n, n).tocsr()


A = matA.toarray()
A = A / deltaS

matA = matA/deltaS
A_w_0_changed = matA.toarray()
A_w_0_changed[0,0] = 2

### dy

diagonalsC = [e5.flatten(), -1 * e2.flatten(), e3.flatten(), -1 * e4.flatten()]
offsetsC = [-(m-1),-1,1,m-1]
matC = spdiags(diagonalsC, offsetsC, n, n)


C = matC.toarray() 
C = C/ (2 * deltax)

matC = matC/ (2 * deltax)

### dx

diagonalsB = [e1.flatten()/  (2 * deltax), -1 * e1.flatten()/ (2 * deltax), e1.flatten()/ (2 * deltax), -1 * e1.flatten()/ (2 * deltax)]
offsetsB = [-(n-m),-m,m, n-m]
matB = spdiags(diagonalsB, offsetsB, n, n)

B = matB.toarray()

In [18]:
tspan = np.arange(0, 20.5, 0.5)
n = 128

## FFT solve

In [20]:


def fft(w0, tspan, n):
    v = 0.001
    nx, ny = n, n
    N = nx * ny
    Lx,Ly = 20,20
    
    ## spectral k values
    kx = (2 * np.pi / Lx) * np.concatenate((np.arange(0,nx/2), np.arange(-nx/2, 0)))
    ky = (2 * np.pi / Ly) * np.concatenate((np.arange(0,ny/2), np.arange(-ny/2, 0)))
    kx[0] = 1e-6
    ky[0] = 1e-6
    
    KX, KY = np.meshgrid(kx,ky)
    K = KX**2 + KY**2
    
    wv = w0.reshape(nx **2,)
    
    
    def fft_rhs(t, w, nx, ny, N, K, matA, matB, matC):
        deltaS = (20/nx) **2
        matA[0,0] = 2/deltaS
        
        wsq = w.reshape(nx,ny)

        wfft = fft2(wsq)
        psi_ft = -wfft / K
        psiV = np.real(ifft2(psi_ft)).reshape((nx **2,))
        
        return ((v * matA.dot(w)) - (matB.dot(psiV)) * (matC.dot(w)) + (matC.dot(psiV)) * (matB.dot(w)))
            
    
    sol_ft = solve_ivp(fft_rhs, (tspan[0],tspan[-1]), wv, method='RK45', t_eval=tspan, args = (nx, ny, N, K, matA, matB, matC))
    return sol_ft.y

In [21]:
## initial conditions & calling A1

v = 0.001
Lx,Ly = 20,20
nx, ny = n, n
N = nx * ny



X = np.linspace(-10,10,n+1)
Y = np.linspace(-10,10,n+1)
x = X[:n]
y = Y[:n]
x1, y1 = np.meshgrid(x, y)

w0 = 1 * np.exp((-x1**2) -((y1**2)/20)) 
wv = w0.reshape(nx **2,)
wrv = np.real(wv)

start_t = time.time()

A1 = fft(w0, tspan, n)

end_t = time.time()
elapsed_time = end_t - start_t

print(elapsed_time)


0.6545441150665283


## A/ b

In [23]:
### A / b


l = 4
dl = 0.5
v = 0.001


X = np.linspace(-10,10,n+1)
Y = np.linspace(-10,10,n+1)

x = X[:n]
y = Y[:n]

x1, y1 = np.meshgrid(x, y)

w = np.exp((-x1**2) -((y1**2)/20))
deltaS = (20/n) **2

### A/ B


def a_b(w, tspan, n):
    
    # matA[0,0] = 2/deltaS
    

    nx, ny = n, n
    N = nx * ny
    wVec = w.reshape(n**2,)
    
    def a_b_rhs(t, w, nx, ny, matA, matB, matC):
        psi_ab = spsolve(matA, w)
        
        return ((v * matA.dot(w)) - (matB.dot(psi_ab)) * (matC.dot(w)) + (matC.dot(psi_ab)) * (matB.dot(w)))
            
   
    sol_ab = solve_ivp(a_b_rhs, (tspan[0],tspan[-1]), wVec, method='RK45', t_eval=tspan, args = (nx, ny, matA, matB, matC))
    return sol_ab.y
       

In [ ]:


start_t = time.time()

A2 = a_b(w, tspan, n)

end_t = time.time()
elapsed_time = end_t - start_t

print(elapsed_time)

## LU decomp

In [ ]:

### lu decomp


l = 4
dl = 0.5
v = 0.001

X = np.linspace(-10,10,n+1)
Y = np.linspace(-10,10,n+1)

x = X[:n]
y = Y[:n]

x1, y1 = np.meshgrid(x, y)

w0_lu = np.exp((-x1**2) -((y1**2)/20))

A[0,0] = 2/deltaS

P, L, U = lu(A)
deltax = 20/n
deltaS = (20/n) **2

def L_U(w0_lu, tspan, n):
    
    deltaS = (20/n) **2
    matA[0,0] = 2/deltaS
    
    wVec_lu = w0_lu.reshape(n**2,)
    # print("wVec_lu", wVec_lu)
    
    def lu_rhs(t, w, P, L, U, matA, matB, matC):
        
        Pb = np.dot(P, w)
        SOL = solve_triangular(L, Pb, lower=True)
        psi_lu = solve_triangular(U, SOL)
        
        return ((v * matA.dot(w)) - (matB.dot(psi_lu)) * (matC.dot(w)) + (matC.dot(psi_lu)) * (matB.dot(w)))
    
    sol = solve_ivp(lu_rhs, (tspan[0],tspan[-1]), wVec_lu, method='RK45', t_eval=tspan, args = (P, L, U, matA, matB, matC))
    # print("sol.y", sol.y.)
   
    
    return sol.y

In [ ]:

start_t = time.time()

A3 = L_U(w0_lu, tspan, n)

end_t = time.time()
elapsed_time = end_t - start_t

print(elapsed_time)

## gmres

In [ ]:
#### gmres

l = 4
dl = 0.5
v = 0.001

X = np.linspace(-10,10,n+1)
Y = np.linspace(-10,10,n+1)

x = X[:n]
y = Y[:n]


tol = 1e-6
maxit = 500
restart = 70

a = matA.toarray()

from scipy.sparse.linalg import LinearOperator, gmres
dia_inv = 1 / np.diagonal(a)  # Inverse of the diagonal (Jacobi preconditioner)
M1 = np.diag(dia_inv)  # Diagonal matrix (Jacobi preconditioner)

# Make sure M1 is a sparse matrix (this step may not be necessary if M1 is already sparse)
M1_sparse = csr_matrix(M1)


x1, y1 = np.meshgrid(x, y)

w0_gr = np.exp((-x1**2) -((y1**2)/20))

def grm_es(w, tspan, n):
    matA[0,0] = 2
    
    A_w_0_changed = matA.toarray()
    A_w_0_changed[0,0] = 2/ deltaS
    
    nx, ny = n, n
    N = nx * ny
    wV = w0_gr.reshape(n**2,)

    iteration_count = 0
    
    def residual_callback(residuals):
        nonlocal iteration_count
        iteration_count += 1
        print(f"Iteration {iteration_count}: Residual norm = {residuals}")
    
    def grm_rhs(t, w, nx, ny, matA, matB, matC):
        
        x0 = np.zeros_like(w)
        psi, flag = gmres(matA, w, rtol=tol, restart=restart, maxiter=maxit , x0=x0)
         
        return ((v * matA.dot(w)) - (matB.dot(psi)) * (matC.dot(w)) + (matC.dot(psi)) * (matB.dot(w)))
            
   
    sol_gm = solve_ivp(grm_rhs, (tspan[0],tspan[-1]), wV, method='RK45', t_eval=tspan, args = (nx, ny, matA, matB, matC))
    return sol_gm.y
    


In [ ]:
start_t = time.time()

A4 = grm_es(w0_gr, tspan, n)

end_t = time.time()
elapsed_time = end_t - start_t

print(elapsed_time)

# print(A4)
# for j in range (9):
#     plt.figure(figsize=(10, 4))
#     plt.subplot(1, 2, 1)
#     plt.imshow(A4[:,j].reshape(n,n), extent=(-10, 10, -10, 10), origin='lower', cmap='viridis')
#     plt.colorbar()
#     plt.title('w0')
#     plt.xlabel('x')
#     plt.ylabel('y')

## bicstab

In [ ]:
#### bicstab
a = matA.toarray()

tol = 1e-6
maxit = 500
restart = 70


x1, y1 = np.meshgrid(x, y)
w0_bc = np.exp((-x1**2) -((y1**2)/20))

def bc_stab(w, tspan, n):
    matA[0,0] = 2
    
    A_w_0_changed = matA.toarray()
    A_w_0_changed[0,0] = 2/ deltaS
    
    nx, ny = n, n
    N = nx * ny
    wv = w0_bc.reshape(n**2,)

    # iteration_count = 0
    
    # def callback(residuals):
    #     nonlocal iteration_count
    #     iteration_count += 1
    #     print(f"Iteration {iteration_count}: Residual norm = {residuals}")
    
    
    def bcs_rhs(t, w, nx, ny, matA, matB, matC):
        
        x0 = np.zeros_like(w)
        psi, flag = bicgstab(matA, w, tol=tol, maxiter=maxit, x0=x0,)
         
        return ((v * matA.dot(w)) - (matB.dot(psi)) * (matC.dot(w)) + (matC.dot(psi)) * (matB.dot(w)))
            
   
    sol_bc = solve_ivp(bcs_rhs, (tspan[0],tspan[-1]), wv, method='RK45', t_eval=tspan, args = (nx, ny, matA, matB, matC))
    return sol_bc.y
    


In [ ]:
start_t = time.time()

A5 = bc_stab(w0_bc, tspan, n)

end_t = time.time()
elapsed_time = end_t - start_t

print(elapsed_time)

# print(A4)
# for j in range (9):
#     plt.figure(figsize=(10, 4))
#     plt.subplot(1, 2, 1)
#     plt.imshow(A5[:,j].reshape(n,n), extent=(-10, 10, -10, 10), origin='lower', cmap='viridis')
#     plt.colorbar()
#     plt.title('w0')
#     plt.xlabel('x')
#     plt.ylabel('y')

In [ ]:
#### bicstab iterations/residuals
a = matA.toarray()

tol = 1e-6
maxit = 500
restart = 70


x1, y1 = np.meshgrid(x, y)
w0_bc = np.exp((-x1**2) -((y1**2)/20))

def bc_stab_res_iter(w, tspan, n):
    matA[0,0] = 2
    
    A_w_0_changed = matA.toarray()
    A_w_0_changed[0,0] = 2/ deltaS
    
    nx, ny = n, n
    N = nx * ny
    wv = w0_bc.reshape(n**2,)

    iteration_count = 0
    residuals_list = []
    
    def callback(residuals):
        nonlocal iteration_count
        iteration_count += 1
        residuals_list.append(residuals)  
        # print(f"Iteration {iteration_count}: Residual norm = {residuals}")
    
    
    def bcs_rhs(t, w, nx, ny, matA, matB, matC):
        
        x0 = np.zeros_like(w)
        psi, flag = bicgstab(matA, w, tol=tol, maxiter=maxit, x0=x0, callback=callback)
         
        return ((v * matA.dot(w)) - (matB.dot(psi)) * (matC.dot(w)) + (matC.dot(psi)) * (matB.dot(w)))
            
   
    sol_bc = solve_ivp(bcs_rhs, (tspan[0],tspan[-1]), wv, method='RK45', t_eval=tspan, args = (nx, ny, matA, matB, matC))
    return residuals_list
    


In [ ]:

res_list = bc_stab_res_iter(w0_bc, tspan, n)
plt.plot(res_list[:1000], label='Residual Norm')
plt.figure(figsize=(8, 6))
# plt.yscale('log')  # Log scale for better visibility of convergence
plt.xlabel('Iteration Number')
plt.ylabel('Residual Norm')
plt.title('Residual Norm vs Iteration')
plt.legend()
plt.show()

## Opp poles

In [ ]:
#### opposite poles

d = 5    # separation distance between vortices
s = 2  # width of the Gaussian
a1 = 1    # amplitude of the positive vortex
a2 = -1


X = np.linspace(-10,10,n+1)
Y = np.linspace(-10,10,n+1)

x = X[:n]
y = Y[:n]

a, b = -d, 0  # position of the positive vortex
c, e = d, 0 

x1, y1 = np.meshgrid(x, y)

w1 = a1 * np.exp(-((x1 - a)**2 + (y1 - b)**2) / (2 * s**2))
w2 = a2 * np.exp(-((x1 - c)**2 + (y1 - e)**2) / (2 * s**2))

opp_w = (w1 + w2)


opp_poles = fft(opp_w, tspan, n)

# for j in range (9):
#     plt.figure(figsize=(10, 4))
#     plt.subplot(1, 2, 1)
#     plt.imshow(opp_poles[:,j].reshape(n,n), extent=(-10, 10, -10, 10), origin='lower', cmap='viridis')
#     plt.colorbar()
#     plt.title('w0')
#     plt.xlabel('x')
#     plt.ylabel('y')

## Same Poles

In [ ]:
#### same charged poles

d = 5    # separation distance between vortices
s = 2  # width of the Gaussian
a1 = 1    # amplitude of the positive vortex
a2 = 1

X = np.linspace(-10,10,n+1)
Y = np.linspace(-10,10,n+1)

x = X[:n]
y = Y[:n]

a, b = -d, 0  # position of the positive vortex
c, e = d, 0 

x1, y1 = np.meshgrid(x, y)

w1 = a1 * np.exp(-((x1 - a)**2 + (y1 - b)**2) / (2 * s**2))
w2 = a2 * np.exp(-((x1 - c)**2 + (y1 - e)**2) / (2 * s**2))

same_w = (w1 + w2)  # + 1j * np.zeros((nx,ny))


same_poles = fft(same_w, tspan, n)
# for j in range (9):
#     plt.figure(figsize=(10, 4))
#     plt.subplot(1, 2, 1)
#     plt.imshow(same_poles[:,j].reshape(n,n), extent=(-10, 10, -10, 10), origin='lower', cmap='viridis')
#     plt.colorbar()
#     plt.title('w0')
#     plt.xlabel('x')
#     plt.ylabel('y')

## Pair Poles

In [ ]:

# Define parameters

d = 5    # Separation between vort
s = 2    # Width of the Gaussian 
a1 = 1   # Amp of the pos vortex
a2 = -1  # Amp of the neg vortex
   # Grid size 
l = 10   # Size of the grid 



# Spatial grid
X = np.linspace(-10, 10, n + 1)  
Y = np.linspace(-10, 10, n + 1)  
x = X[:n]  
y = Y[:n]  

# Positions of first pair 
a, b = -d, 0  
c, e = d, 0   

# Create meshgrid for X and Y
x1, y1 = np.meshgrid(x, y)

# Create the Gaussian vortices
w1 = a1 * np.exp(-((x1 - a)**2 + (y1 - b)**2) / (2 * s**2))  # Positive vortex 1
w2 = a2 * np.exp(-((x1 - c)**2 + (y1 - e)**2) / (2 * s**2))  # Negative vortex 1


# Position for second pair 
f, g = -d, 5  
h, i = d, 5   

w3 = a1 * np.exp(-((x1 - f)**2 + (y1 - g)**2) / (2 * s**2))  # Positive vortex 2
w4 = a2 * np.exp(-((x1 - h)**2 + (y1 - i)**2) / (2 * s**2))  # Negative vortex 2

# Combine 
pair_w = w1 + w2 + w3 + w4

pair_poles = fft(pair_w, tspan, n)
# for j in range (9):
#     plt.figure(figsize=(10, 4))
#     plt.subplot(1, 2, 1)
#     plt.imshow(pair_poles[:,j].reshape(n,n), extent=(-10, 10, -10, 10), origin='lower', cmap='viridis')
#     plt.colorbar()
#     plt.title('w0')
#     plt.xlabel('x')
#     plt.ylabel('y')

## many rand vortices

In [ ]:
#### many random vortices

           
l = 10            
num_vortices = 10 # Number of vortices

# Generate random pos
x_pos = np.random.uniform(-l, l, num_vortices)
y_pos = np.random.uniform(-l, l, num_vortices)

# Generate random strength
strengths = np.random.uniform(-1, 1, num_vortices)

# Generate random ellipticities 
ellipticities = np.random.uniform(0.5, 2, num_vortices)  

# Generate random widths (
widths = np.random.uniform(1, 3, num_vortices)

# Initialize the vortex field to zero
vortex_field = np.zeros((n, n))

# Create vortices by adding Gaussian profiles at random positions
for i in range(num_vortices):
    # Random vortex parameters
    x0 = x_pos[i]
    y0 = y_pos[i]
    strength = strengths[i]
    aspect_ratio = ellipticities[i]
    width = widths[i]
    
    # Elliptical Gaussian profile (adjust aspect ratio)
    gaussian = np.exp(-((x1 - x0)**2 + ((y1 - y0)/aspect_ratio)**2) / (2 * width**2))
    
    # Add the vortex contribution to the field
    vortex_field += strength * gaussian
    
many_vort = fft(vortex_field, tspan, n)
# for j in range (9):
#     plt.figure(figsize=(10, 4))
#     plt.subplot(1, 2, 1)
#     plt.imshow(many_vort[:,j].reshape(n,n), extent=(-10, 10, -10, 10), origin='lower', cmap='viridis')
#     plt.colorbar()
#     plt.title('w0')
#     plt.xlabel('x')
#     plt.ylabel('y')

## MOVIES

In [ ]:
colors = [(0, "blue"),(0.5, "white"), (1, "purple")]  # Start with blue, end with purple
custom_cmap = LinearSegmentedColormap.from_list("blue_to_purple", colors)


def heatmap_animation (data, cmap ='viridis', save_as=None, fps=10):  
    # Initialize the figure and axis
    fig, ax = plt.subplots()
    heatmap = ax.imshow(data[:, :, 0], cmap = custom_cmap, origin='lower')
    plt.colorbar(heatmap, ax=ax)

    def update(frame):
        heatmap.set_data(data[:, :, frame])  # Update the heatmap for each frame
        return [heatmap]

    # Create the animation
    ani = FuncAnimation(fig, update, frames=data.shape[2], blit=True, repeat=True)
   
    if save_as:
        # Use Pillow for GIF
        ani.save(save_as, writer='pillow', fps=fps)

    # Display the animation
    display(HTML(ani.to_jshtml()))
   
    # Close the figure to prevent showing the last frame as a static plot
    plt.close(fig)

In [ ]:
### movies


n = 128

heatmap_animation(opp_poles.reshape(n,n, len(tspan)), 'viridis', None, 10 )

In [ ]:
heatmap_animation(pair_poles.reshape(n,n, len(tspan)), 'viridis', None, 10 )

In [ ]:
### movies
n = 128
colors = [(0, "black"),(0.5, "green"), (1, "white")]  # Start with blue, end with purple
custom_cmap = LinearSegmentedColormap.from_list("blue_to_purple", colors)


heatmap_animation(many_vort.reshape(n,n, len(tspan)), 'viridis', None, 10 )

In [ ]:
colors = [(0, "lightblue"), (1, "black")]  # Start with blue, end with purple
custom_cmap = LinearSegmentedColormap.from_list("blue_to_purple", colors)

heatmap_animation(same_poles.reshape(n,n, len(tspan)), 'viridis', None, 10 )

In [ ]:

heatmap_animation(A1.reshape(n,n, len(tspan)), 'viridis', None, 10 )